In [11]:
# add noise independently to each attribute
# use same epsilon and delta 
# use binary mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#binary-mechanism) for sex
# use geometric mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#geometric-mechanisms) for age
# use binary mechanism (https://diffprivlib.readthedocs.io/en/latest/modules/mechanisms.html#binary-mechanism) to select real value or representative random value for race

In [17]:
import numpy as np
import matplotlib.pyplot as plt
from diffprivlib import tools as dp
from diffprivlib.mechanisms import Binary
from diffprivlib.mechanisms import Geometric
import pandas as pd
import random

#from all_us distribution
real_order = [58, 59, 60, 52, 54, 51, 48, 32, 42, 38, 36, 40, 34, 33, 43, 23, 16, 41, 29, 24, 15, 22, 28, 9, 21, 11, 4, 8, 6, 3, 1, 0, 5, 2, 10, 7, 14, 19, 12, 18, 17, 20, 13, 25, 27, 47, 31, 30, 26, 35, 56, 44, 37, 50, 45, 39, 57, 62, 46, 49, 55, 53, 61]

races = ['Total!!Population of one race!!White alone',
       'Total!!Population of one race!!Black or African American alone',
       'Total!!Population of one race!!American Indian and Alaska Native alone',
       'Total!!Population of one race!!Asian alone',
       'Total!!Population of one race!!Native Hawaiian and Other Pacific Islander alone',
       'Total!!Population of one race!!Some Other Race alone',
       'Total!!Two or More Races!!Population of two races!!White; Black or African American',
       'Total!!Two or More Races!!Population of two races!!White; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!White; Asian',
       'Total!!Two or More Races!!Population of two races!!White; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!White; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Asian',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of two races!!Asian; Some Other Race',
       'Total!!Two or More Races!!Population of two races!!Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; American Indian and Alaska Native',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Asian',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Black or African American; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!White; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!White; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of three races!!Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Asian',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; American Indian and Alaska Native; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Black or African American; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!White; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of four races!!American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Asian; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; American Indian and Alaska Native; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; Black or African American; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!White; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of five races!!Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race',
       'Total!!Two or More Races!!Population of six races!!White; Black or African American; American Indian and Alaska Native; Asian; Native Hawaiian and Other Pacific Islander; Some Other Race']

race_dict1 = {}
race_dict2 = {}

for i in range(len(races)):
    race_dict1[i] = races[i]
    race_dict2[races[i]] = i

tenuredict = {0: 'Own', 1:'Rent'}
tenuredict2 = {'Own': 0, 'Rent':1}

In [18]:
##########################################################################################################################
#CREATE BLOCK DATA
#input: csv containing block data 
#output: dataframe containing block data for age, sex, race, etc
##########################################################################################################################

def create_block_data(path):
    df = pd.read_csv(path)
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
    return df

# df = create_block_data('../homemade_data/almeda.csv')
# orig_df = df.copy()

In [27]:
##########################################################################################################################
#MAKE DIFFERENTIALLY PRIVATE
#input: 1. dataframe dframe containing block data 2. race_data containing representative race vals 3. epsilon val (float)
#output: differentially private representation of dataframe df
##########################################################################################################################

def make_differentially_private(dft,  ep):
    
    dframe = dft.copy()
    b_sex = Binary(epsilon=ep, value0="m", value1="f")
    geo_age = Geometric(epsilon=ep, sensitivity=115)
#     b2 = Binary(epsilon=ep, value0="true value", value1="fake value") I don't think we use this -Miranda
    geo_race = Geometric(epsilon=ep, sensitivity=62)
    geo_size = Geometric(epsilon=ep, sensitivity=3)
    b_tenure = Binary(epsilon=ep, value0="Own", value1="Rent")
    for (columnName, columnData) in dframe.iteritems():

        if columnName == "race":
            i = 0
            for g in columnData.values:
                idx = real_order.index(g)                
                blah = geo_race.randomise(idx)%63
                h = race_dict1[real_order[blah]]
                dframe.at[i, 'race'] = h
                
                h = geo_race.randomise(g)%63
                dframe.at[i, 'race'] = h
                i +=1
    return dframe



In [33]:
# Make dprun2 with one dataset per swap rate (for testing purposes)
counties = ['alameda']
for county in counties:
    orig_df = create_block_data('../homemade_data/'+county+'.csv')
    for index, row in orig_df.iterrows():
        d = row['race']
        orig_df.at[index, 'race'] = races.index(d)
    csv_orig_data = orig_df.to_csv('dp_runs/'+county+'/orig_data.csv', index = True)

    epsilon = np.arange(1, 7, 1, float)

    for e in epsilon:
        print(str(e)[0:4])
        df = orig_df.copy()
        dframe = make_differentially_private(df, e)
#         filename = "dp_runs/"+county+"/dprun_"+str(e)[0:4]+".csv"
        filename = "dp_runs/"+county+"/dprun_"+str(e)[0:4]+"_0.csv"
        csv_orig_data = dframe.to_csv(filename, index = True)

1.0
2.0
3.0
4.0
5.0
6.0
